## Routenplanung

## Set Path for Imports

In [25]:
import sys

sys.path.insert(0, "./work")

## Imports

In [26]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from pyspark.sql import SparkSession, DataFrame
import folium
import requests
import polyline

from lib.utils import get_coordinates, create_distance_matrix

## Setup Spark

In [27]:
spark = SparkSession.builder \
    .appName("SmartLitter") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://db:5432/litter_db"
connection_properties = {
    "user": "root",
    "password": "pwd123",
    "driver": "org.postgresql.Driver"
}

## Load Data

In [28]:
df = spark.read.jdbc(url=jdbc_url, table="public.litter_bin_geoposition", properties=connection_properties)

df = df.sort(df.geom_point)

## Create Data Model

In [29]:
def create_data_model(coordinates: list[tuple[float, float]]) -> dict:
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = create_distance_matrix(coordinates)
    data["coordinates"] = coordinates
    data["num_vehicles"] = 1
    data["depot"] = 0
    data["shape"] = (len(data["distance_matrix"]), len(data["distance_matrix"][0]))
    return data

## Get Coordinates and Create Data Model

In [30]:
coordinates: list[tuple[float, float]] = get_coordinates(df.limit(60))

data: dict = create_data_model(coordinates)

## Solve the Problem

In [31]:
coords: list[tuple[float, float]] = []

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} meters")
    index = routing.Start(0)
    plan_output = "Route for vehicle 1:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        coords.append(data["coordinates"][manager.IndexToNode(index)])
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    coords.append(data["coordinates"][manager.IndexToNode(index)])
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}meters\n"
 
 
def main():
    """Entry point of the program."""
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]
 
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(manager, routing, solution)
 
 
if __name__ == "__main__":
    main()

Objective: 23286 meters
Route for vehicle 1:
 0 -> 31 -> 19 -> 9 -> 20 -> 45 -> 7 -> 21 -> 52 -> 60 -> 22 -> 25 -> 11 -> 53 -> 56 -> 55 -> 37 -> 39 -> 46 -> 18 -> 26 -> 38 -> 33 -> 50 -> 23 -> 51 -> 27 -> 48 -> 44 -> 54 -> 34 -> 36 -> 32 -> 5 -> 59 -> 40 -> 41 -> 28 -> 42 -> 30 -> 2 -> 16 -> 8 -> 17 -> 58 -> 24 -> 4 -> 15 -> 47 -> 35 -> 43 -> 1 -> 12 -> 6 -> 10 -> 13 -> 49 -> 14 -> 3 -> 29 -> 57 -> 0


## Display Route on Map

In [32]:
# Format coordinates for the OSRM API
coordinate_string = ";".join([f"{lat},{lng}" for lng, lat in coords])
 
# Request route from OSRM API
url = f"http://router.project-osrm.org/route/v1/driving/{coordinate_string}?overview=full"
response = requests.get(url)
route_data = response.json()
 
# Extract and decode the polyline of the route
encoded_polyline = route_data['routes'][0]['geometry']
decoded_route = polyline.decode(encoded_polyline)
 
# Create a Folium map centered at the first coordinate
start_lat, start_lng = coords[0]
m = folium.Map(location=[start_lat, start_lng], zoom_start=13)
 
# Add the route as a polyline to the map
folium.PolyLine(
    locations=decoded_route,  # Use the decoded coordinates
    color='blue',
    weight=4,
    opacity=0.7
).add_to(m)
 
 
 
# Add a marker for the start point
folium.Marker(
    location=coords[0],
    popup="Start/Ende",
    icon=folium.Icon(color="green", icon="play"),
).add_to(m)
 
# Add markers for intermediate waypoints
for point in coords[1:-1]:
    folium.Marker(
        location=point,
        popup="Kübel",
        icon=folium.Icon(color="blue", icon="map-marker"),
    ).add_to(m)
 
# Add a marker for the end point
 
 
# Save the map to an HTML file or display it in a Jupyter notebook
m.save("data/osrm_route_map.html")